# Answering Business Questions using SQL

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
--finding the total number of tracks that were sold within the United States

SELECT COUNT(quantity) total_tracks_sold_usa FROM invoice_line il
INNER JOIN invoice i ON i.invoice_id = il.invoice_id
INNER JOIN customer c ON c.customer_id = i.customer_id
WHERE c.country = "USA"

 * sqlite:///chinook.db
Done.


total_tracks_sold_usa
1051


In [4]:
%%sql
--now, we need to dig deeper and break down the total number of tracks sold in the usa by genre and market share
--Write a query that returns each genre, with the number of tracks sold in the USA: in absolute numbers; percentages

WITH 
    usa AS 
    (
    SELECT il.*, i.*, c.* FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    WHERE c.country = "USA"
    )
    
SELECT g.name genre, SUM(quantity) tracks_sold_usa, CAST(count(quantity) as float)/ 
    ( 
    SELECT COUNT(*) FROM usa 
    ) percentage_sold       
FROM usa
INNER JOIN track t ON t.track_id = usa.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


genre,tracks_sold_usa,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


#### If a music shop in the USA wanted to restock on inventory, they would be wise to buy a relatively larger share of Rock, Alternative & Punk, and Metal albums in comparison to other genres.

---

In [5]:
%%sql
--Write a query that finds the total $ amount of sales assigned to each sales support agent within the company
--Compare the results of the sales support agents' performances

WITH support_agent_info AS
    (
    SELECT 
        e.employee_id,
        e.first_name || " " || e.last_name employee_name,
        e.hire_date,
        ROUND(SUM(i.total), 2) total_sales
    FROM employee e
    INNER JOIN customer c ON c.support_rep_id = e.employee_id
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY 1, 2
    )
    
SELECT * FROM support_agent_info sai
GROUP BY 1

 * sqlite:///chinook.db
Done.


employee_id,employee_name,hire_date,total_sales
3,Jane Peacock,2017-04-01 00:00:00,1731.51
4,Margaret Park,2017-05-03 00:00:00,1584.0
5,Steve Johnson,2017-10-17 00:00:00,1393.92


#### While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

---

In [6]:
%%sql
--Write a query that collates data on purchases from different countries
--Where a country has only one customer, they will be sorted into an "other" group
--The results are sorted by the total sales in descending order, with the "other" group at the very bottom

WITH 
    country_list AS
    (
     SELECT
        CASE
            WHEN
                (
                SELECT COUNT(*) FROM customer
                WHERE country = c.country
                ) = 1
                THEN "Other"
                ELSE c.country
            END as country,
        c.customer_id,
        il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
     ORDER BY country ASC
    )
           
SELECT 
    country,
    total_customers,
    total_sales,
    average_order,
    customer_lifetime_value

FROM 
    (
     SELECT
        country,
        count(distinct customer_id) total_customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_list
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


#### If we were debating whether to spend money on an advertising campaign, we would have to be cautious about investing too much money. There quantity of data from each of these countries is relatively low; with a small sample size, we need to be wary of the reliability of the data. The best approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

---

In [9]:
%%sql
--query to see the album_id and number of purchases
--since there are no 0s, that means that every album in database was bought at least once

WITH 
    num_of_album_tracks_sold AS 
        (
        SELECT 
            album_id,
            COUNT(track_id) num_tracks_sold
        FROM track
        GROUP BY 1
        )
    
SELECT album_id, num_tracks_sold FROM num_of_album_tracks_sold

 * sqlite:///chinook.db
Done.


album_id,num_tracks_sold
1,10
2,1
3,3
4,8
5,15
6,13
7,12
8,14
9,8
10,14


In [8]:
%%sql
--Write a query that categorizes each invoice as either an album purchase or not
--Within the query, calculate the number of invoices and percentage of invoices

WITH album_or_indiv AS
    (
    SELECT
        CASE
            WHEN COUNT(distinct album_id) = 1 and COUNT(il.track_id)> 3 THEN "album"
            ELSE "individual"
        END purchase_type,
        t.album_id,
        t.name,
        il.invoice_id
    FROM invoice_line il
    INNER JOIN track t ON t.track_id=il.track_id
    GROUP BY 4
    )

SELECT
    CASE
        WHEN purchase_type="album" THEN "yes"
        WHEN purchase_type="individual" THEN "no"
        END album_purchase,
    COUNT(invoice_id) total_type,
    CAST(COUNT(album_id) as float)/
            (
             SELECT COUNT(*) FROM invoice
            ) percentage
    FROM album_or_indiv ai
GROUP BY purchase_type

 * sqlite:///chinook.db
Done.


album_purchase,total_type,percentage
yes,132,0.21498371335504887
no,482,0.7850162866449512


#### Since album purchases account for nearly 21.5% of purchases, I would recommend against purchasing only select tracks from record companies;  21.5% of purchases is over one fifth of total revenue.